In [1]:
from __future__ import print_function
import keras # расскоментируйте эту строку, чтобы начать обучение
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import matplotlib.pyplot as plt

import os

In [2]:
import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [3]:
# установка параметров нейросети
batch_size = 320 
num_classes = 10
epochs = 15
data_augmentation = False 
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [4]:
# разделение тренировочной и тестовой выборки
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

x_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


In [5]:
# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
%%time
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(128, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# инициализация RMSprop optimizer
#opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='Adam', #opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation в реальном времени')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=0, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0., 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)

    # запуск data augmentation через fit
    #datagen.fit(x_train)

    # запуск data augmentation через fit_generator
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

Не используется data augmentation
Epoch 1/15
157/157 [==============================] - 339s 2s/step - loss: 1.7346 - accuracy: 0.3619 - val_loss: 1.3519 - val_accuracy: 0.5121
Epoch 2/15
157/157 [==============================] - 113s 723ms/step - loss: 1.2942 - accuracy: 0.5382 - val_loss: 1.1361 - val_accuracy: 0.5984
Epoch 3/15
157/157 [==============================] - 113s 721ms/step - loss: 1.0874 - accuracy: 0.6134 - val_loss: 0.9998 - val_accuracy: 0.6515
Epoch 4/15
157/157 [==============================] - 113s 718ms/step - loss: 0.9719 - accuracy: 0.6601 - val_loss: 0.8724 - val_accuracy: 0.7024
Epoch 5/15
157/157 [==============================] - 113s 722ms/step - loss: 0.8634 - accuracy: 0.6982 - val_loss: 0.7788 - val_accuracy: 0.7290
Epoch 6/15
157/157 [==============================] - 113s 717ms/step - loss: 0.7824 - accuracy: 0.7262 - val_loss: 0.7454 - val_accuracy: 0.7398
Epoch 7/15
157/157 [==============================] - 113s 720ms/step - loss: 0.7313 - accura

In [7]:
# сохранение модели и весов
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('сохранить обученную модель как %s ' % model_path)

сохранить обученную модель как C:\Users\Pavel_Nazarenko\Intro_to_neuro_Jupyter\saved_models\keras_cifar10_trained_model.h5 


In [8]:
# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 6s 18ms/step - loss: 0.6176 - accuracy: 0.7922
Test loss: 0.6175854206085205
Test accuracy: 0.7922000288963318


При увеличении количества эпох, результаты улучшаются, но до определённого момента, затем точность не растет
При увеличении batch_size, возрастает скорость (расчёт вёлся на GPU), но падает точность.
augmentation очень медленный. можно пробовать его для улучшения качества, но нужны для этого другие мощности.
метод оптимизации Adam лучше.
Увеличил число нейронов - качество улучшилось
Для MNIST, CIFAR100 и IMAGENET необходимо поменять число входных и выходных нейронов, т.к. размеры исходных картинок и количество искомых категорий в этих датасетах различное.